# 大型网站的架构演进

    单机负载告警 -- 数据库与应用分离
    应用服务器负载告警 -- 如何让应用服务器走向集群
        引入负载均衡设备
        解决应用服务器变为集群后的Session问题:
            Cookie -- 禁用Cookie,一般做法 - 把会话标识放到URL的参数中.
            多服务器Session共享问题,解决方案 ->
                
多服务器Session共享问题,解决方案:

1. Session Sticky : 控制负载均衡器,让同一个会话的请求都在同一个web服务器上处理.存在如下问题:**
    1. 如果有一台Web服务器宕机或者重启,那么这台机器上的会话数据会丢失.如果会话中有登陆状态数据,那么用户就要重新登陆
    2. 会话标识时应用层的信息,那么负载均衡器要将同一个会话的请求都保存到同一个Web服务器上的话,就需要进行应用层解析,这个开销比传输层的交换要大
    3. 负载均衡器变为了一个有状态的节点,要将会话保存到具体Web服务器的映射.和无状态的节点相比,内存消耗会更大,容灾方面会更麻烦.
2. Session Replication : 不再要求负载均衡器来保证同一个会话的多次请求必须到同一个Web服务器上,而是Web服务器之间则增加了会话数据的同步.存在如下问题:
    1. 同步Session数据造成了网络带宽的开销
    2. 每台Web服务器都要保存所有的Session数据,造成整个集群中每台机器用于保存Session数据的内容占用很严重
3. Session数据集中存储 : 把Session数据集中存储起来,然后不同Web服务器从同样的地方来获取Session.保证了不同Web服务器上读到的Session数据都是一样的,而存储Session数据的具体方式,可以使用数据库,也可以使用其他分布式存储系统.存在问题:**
    1. 读写Session数据引入了网络操作, 相对于本机的数据存取,存在时延和不确定性,但通信基本都是发生在内网,影响不大
    2. 如果集中存储Session的机器或者集群有问题,就会影响应用
4. Cookie Based : 通过Cookie来传递Session数据.不足:
    1. Cookie长度的限制
    2. 安全性
    3. 带宽消耗
    4. 性能影响

读写分离:

1. 采用数据库作为读库 : 
    带来两个问题:数据复制问题/应用对于数据源的选择问题
    数据怎么复制到读库的问题: 数据库系统一般都提供了数据复制的功能,可以直接使用数据库系统的自身机制.需要考虑的问题:1-数据复制时延问题;2-复制过程中数据的源和目标之间的映射关系及过滤条件的支持问题.
    对于应用而言,增加一个读库对结构变化有一个影响,即应用需要根据不同的情况来选择不同数据库源(配置多数据源,基于AOP匹配mybatis中mapper不同的方法).写操作要走主库,事务中的读也要走主库,也要考虑备库数据相对于主库数据的延迟.即即便是不在事务中的读,考虑到备库的数据延迟,不同业务下的选择也会有差异.
2. 搜索引擎其实是一个读库
    首先是需要根据被搜索的数据来构建索引.(什么数据应该走索引,什么数据应该走数据库)
    搜索集群的使用方式和读库的使用方式是一样的,只是构建索引的过程基本都是需要我们自己来实现.
    搜索系统构建索引的方式:
        按照全量/增量划分 - 全量方式用于第一次建立索引,而增量方式用于在全量的基础上持续更新索引.注意:增量构建索引的挑战非常大,一般会加入每日的全量作为补充.
        按照实时/非实时划分 - 体现在索引更新的时间上,倾向于实时的方式,存在非实时方式,主要考虑到对数据源头的保护
3. 加速数据读取的利器 - 缓存
    数据缓存: 缓存系统一般是用来保存和查询键值对的.一般在缓存中放的是热数据而不是全部数据.
    页面缓存: ESI(Edge Side Includes，页面片段缓存)语言规范.
    对于使用缓存来加速数据读取的情况,关键指标-缓存命中率.此外,数据的分布与更新策略也需要结合具体的场景来考虑.从分布上来说,主要考虑的问题是需要有机制去避免局部的热点,并且缓存服务器扩容或者缩容要尽量平滑(推荐一致性Hash).而在缓存的数据的更新上,会有定时失效/数据变更时失效和数据变更时更新的不同选择.

分布式存储系统:

1. 常见的分布式存储系统:分布式文件系统/分布式Key-Value系统和分布式数据库.
    1. 分布式文件系统就是在分布式环境中由多个节点组成的功能与单机文件系统一样的文件系统,它是弱格式的,内容的格式需要使用者自己来组织
    2. 分布式Key-Value系统相对分布式文件系统会更加格式化一些
    3. 分布式数据库是最格式化的方式
2. 具体使用场景-分布式存储系统通过集群提供了一个高容量/高并发访问/数据冗余容灾的支持
    1. 分布式文件系统解决小文件和大文件的存储问题
    2. 分布式Key-Value系统提供高性能的半结构化的支持
    3. 分布式数据库提供一个支持大数据/高并发的数据库系统

数据库拆分    
    
1. 专库专用,数据垂直拆分-把数据库中不同业务数据分到不同的数据库中
    带来的影响:应用需配置多个数据源,每个数据库连接池的隔离
    -> 如何处理原来单机中跨业务的事务:
        1. 使用分布式事务,性能要明显低于之前的单机事务
        2. 去掉事务或者不去追求强事务支持,则原来在单库中可以使用的表关联的查询需要改变实现方式
2. 垂直拆分后的单机遇到瓶颈,数据水平拆分 - 把同一个表的数据拆到多个数据库中
    带来的影响:
        首先,访问用户信息的应用系统需要解决SQL路由的问题,因为现在用户信息分在了多个数据库中,需要在进行数据库操作时了解需要操作的数据在哪里
        此外,主键的处理变得不同.需要调整主键生成策略,不能单纯的Sequence或者自增字段
        最后,多数据库返回的数据处理分页问题,需用新策略
        
拆分应用

1. 根据业务的特性把应用拆开 - 造成多个系统重复代码
2. 服务化之路
    1. 业务功能之间的访问不仅是单机内部的方法调用,引入了远程服务调用
    2. 共享的代码不再散落不同的应用,这些实现被放在各个服务中心
    3. 数据库连接策略改变,将与数据库的交互工作放到服务中心,前端的web应用更加注重与浏览器交互的工作,不需关注业务逻辑.
    4. 通过服务化,无论是前端web应用还是服务中心都可以由固定小团队来维护,更容易保持系统稳定性

消息中间件

    Message-oriented middleware(MOM) is software infrastructure focused on sending and receiving messages between distributed systems. 面向消息的系统(消息中间件)是在分布式系统中完成消息的发送和接收的基础软件.
    好处:异步/解耦.